# Strong TMA / M1M3 shaking event occuring on 2023-06-28 around 1 am

In this notebbok we study a peculiar event that occured during the night 2023-06-27 / 2023-06-28 around 1 am and that was reported by Bruno C. Quint
The M1M3 test log file can be found here: https://confluence.lsstcorp.org/display/LSSTCOM/23.06.27+-+M1M3+Test+Log

This notebook is associated to [SITCOM-784](https://rubinobs.atlassian.net/browse/SITCOM-784)

The details of the analysis performed in this notebook are described in the Technical Note SITCOMTN-131

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

%load_ext lab_black

In [ ]:
import os.path

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import h5py
import numpy as np

from astropy.time import Time
from astropy.time import TimeDelta
from astropy import units as u
import matplotlib.dates as mdates
from matplotlib.ticker import FormatStrFormatter
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
from matplotlib import ticker
from scipy.signal import stft
from scipy.signal import find_peaks_cwt, find_peaks
from scipy import signal
from scipy.fft import fft, rfft, fftfreq, rfftfreq


from lsst_efd_client import EfdClient
from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState, TMAEvent
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient
from lsst.ts.xml.enums import MTM1M3, MTMount

In [ ]:
# Utility functions

key_m1m3_dict = {
    "1 X": "m1m3_x_1",
    "1 Y": "m1m3_y_1",
    "1 Z": "m1m3_z_1",
    "2 X": "m1m3_x_2",
    "2 Y": "m1m3_z_2",  # note these two have been
    "2 Z": "m1m3_y_2",  # switched pending SUMMIT-7911
    "3 X": "m1m3_x_3",
    "3 Y": "m1m3_y_3",
    "3 Z": "m1m3_z_3",
}
key_m2_dict = {
    "1 X": "m2_x_1",
    "1 Y": "m2_y_1",
    "1 Z": "m2_z_1",
    "2 X": "m2_x_2",
    "2 Y": "m2_z_2",
    "2 Z": "m2_y_2",
    "3 X": "m2_x_3",
    "3 Y": "m2_z_3",
    "3 Z": "m2_y_3",
    "4 X": "m2_x_4",
    "4 Y": "m2_y_4",
    "4 Z": "m2_z_4",
    "5 X": "m2_x_5",
    "5 Y": "m2_z_5",
    "5 Z": "m2_y_5",
    "6 X": "m2_x_6",
    "6 Y": "m2_z_6",
    "6 Z": "m2_y_6",
}


def vms_data_to_pandas(filename, vms_type, begin_time=None, end_time=None):
    """
    Converts VMS data in the given HDF5 file to a Pandas DataFrame.

    Args:
    filename: Path to the HDF5 file containing the VMS data.
    vms_type: The type of VMS data in the file. Must be "m1m3", "m2", or
      "rotator".
    begin_time: The start time of the data to include in the DataFrame. If None,
      all data will be included.
    end_time: The end time of the data to include in the DataFrame. If None, all
      data will be included.

    Returns:
    A Pandas DataFrame containing the VMS data.
    """
    if vms_type == "m1m3":
        key_dict = key_m1m3_dict
    elif vms_type == "m2":
        key_dict = key_m2_dict
    elif vms_type == "rotator":
        raise NotImplementedError
    else:
        raise ValueError("vms_type must be m1m3,m2, or rotator")

    f = h5py.File(filename, "r")
    times = f["timestamp"][::1]
    dkeys = "XYZ"

    data_dict = {}
    if (begin_time is not None) & (end_time is not None):
        sel = (times > begin_time) & (times < end_time)
    else:
        sel = np.ones(times.size).astype(bool)
    data_dict["times"] = times[sel]
    for key in key_dict.keys():
        # multiply values stored in hdf5 files by 2 in order to convert the acceleration values to mg
        data_dict[key_dict[key]] = f[key][::1][sel] * 2.0
    data_frame = pd.DataFrame(data_dict)
    for j in np.arange(int(len(key_dict) / 3)) + 1:
        data_frame[f"total_{j}"] = np.linalg.norm(
            data_frame[[f"{vms_type}_{i}_{j}" for i in ["x", "y", "z"]]].values, axis=1
        )

    return data_frame


def get_freq_psd(vals, timestep):
    """
    Calculates the frequency power spectrum of a signal.

    Args:
        vals (np.array): The signal values.
        timestep (float): The time step between samples.

    Returns:
        tuple: The frequencies and power spectral density.
    """

    # Remove the mean from the signal.

    meanval = np.mean(vals)
    signal = vals - meanval

    # Calculate the length of the signal.

    N = len(signal)

    # Calculate the power spectral density.

    psd = np.abs(np.fft.rfft(np.array(signal) * 1)) ** 2

    # Calculate the frequencies.

    frequencies = np.fft.rfftfreq(N, timestep)
    return (frequencies, psd)


def get_psd_and_dsd_for_vms(vals, timestep, min_freq=1, g=False):
    """
    Calculate the PSD and DSD from VMS data, and total displacement from DSD.

    Parameters:
    - vals (array-like): VMs in m/s^2 or milli-g if 'g' is True.
    - timestep (float): Time step between measurements in seconds.
    - min_freq (float, optional): Minimum frequency for calculations.
                                  Default is 1 Hz.
    - g (bool, optional): True if 'vals' are in milli-g units.
                          Default is False.

    Returns:
    - psds_df (DataFrame): DataFrame with frequencies ('freq'), acceleration
                           PSD ('accel_psd'), displacement PSD ('disp_psd'),
                           and cumulative displacement PSD ('int_disp').
    - total_displacement (float): Total displacement from vibration data.

    Note:
    PSD adjusted to m/s^2/Hz. 'vals' converted to m/s^2 if 'g' is True.
    """
    if g:
        vals = 1e-3 * 9.8 * vals

    # freq, accel_psd = signal.periodogram(vals, 1/timestep)
    # print("using scipy")
    freq, accel_psd = get_freq_psd(vals, timestep)

    sel = freq > min_freq
    freq = freq[sel]
    accel_psd = accel_psd[sel]

    accel_psd = accel_psd * np.mean(np.diff(freq))

    disp_psd_sq = accel_psd / ((2 * np.pi * freq) ** 4)

    int_displace_psd = np.sqrt(np.cumsum(disp_psd_sq))
    total_displacement = np.sqrt(np.sum(disp_psd_sq))

    psds_df = pd.DataFrame(
        {
            "freq": freq,
            "accel_psd": accel_psd,
            "disp_psd": np.sqrt(disp_psd_sq),
            "int_disp": int_displace_psd,
        }
    )

    return psds_df, total_displacement

In [ ]:
# Create a directory to save plots
plot_dir = "./plots"
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

In [ ]:
# Acquisition date of VMS data
vms_date = "2023-06-28"

vms_top_dir = "/sdf/scratch/users/b/boutigny/vmsdata"
year, month = vms_date.split("-")[0:2]

# Directory containing VMS data
vms_dir = os.path.join("/scratch/users/b/boutigny/vmsdata", year, month)

# Check if a parquet file exists
vms_m1m3_parquet_filename = os.path.join(vms_dir, "M1M3-" + vms_date + "T00:00.parquet")
if os.path.isfile(vms_m1m3_parquet_filename):
    print(f"Reading VMS data from parquet file:{vms_m1m3_parquet_filename}")
    vms_m1m3_data = pd.read_parquet(vms_m1m3_parquet_filename)
else:
    # read hdf file and populate pandas dataframe
    vms_m1m3_hdf_filename = os.path.join(vms_dir, "M1M3-" + vms_date + "T00:00.hdf")
    print(f"Reading VMS data from hdf5 file:{vms_m1m3_hdf_filename}")
    vms_m1m3_data = vms_data_to_pandas(vms_m1m3_hdf_filename, vms_type="m1m3")
    # Remove entries with null timestamps
    vms_m1m3_data = vms_m1m3_data[vms_m1m3_data["times"] > 0]
    # Reformat timestamps
    vms_m1m3_data["times"] = Time(vms_m1m3_data["times"], format="unix").datetime

In [ ]:
%matplotlib inline
# First look at the raw VMS data - We plot only 1 sensor and 1 axis because the full dataset is very large and requires too much memory
# to be displayed
fig, ax = plt.subplots(1, 1, dpi=125, figsize=(8, 4))
key = "m1m3_x_3"
ax.plot(vms_m1m3_data["times"], vms_m1m3_data[key])
ax.set(ylabel="Acceleration (1E-3 g)", xlabel="Time", title=f"{vms_date} - {key}")
ax.xaxis.set_major_formatter(DateFormatter("%H-%M-%S"))
plt.setp(ax.get_xticklabels(), rotation=45)
fig.tight_layout()
fig.savefig(f"{plot_dir}/VMS-accel-{vms_date}.png")

In the previous plot we see some vibrations ocuring at different times all over the time period. The amplitude of the vibrations are not especially large.

In the following, we will investigate ore into details the time period around 1 am.

In [ ]:
# Retrieve TMA events corresponding to the VMS acquisition date
# As VMS and TMA do not cover the same time window, we need to get the TMA events corresponding to both
# dayObs and the day before dayObs

dayObs = int(vms_date.replace("-", "", 2))
vms_time = Time(vms_date + "T00:00:00")
delta_t = TimeDelta(1, format="jd")
day_before = int(str(vms_time - delta_t)[0:10].replace("-", "", 2))
eventMaker = TMAEventMaker()
events = eventMaker.getEvents(day_before)
events = events + eventMaker.getEvents(dayObs)

# Get lists of slew events
slews = [e for e in events if e.type == TMAState.SLEWING]
print(f"Found {len(slews)} slews")

In [ ]:
# Filter the list of slews in order to keep the ones that are fully contained within the day corresponding to the VMS file.
date_min = Time(f"{vms_date} 00:00:00.00").unix
date_end = Time(f"{vms_date} 23:59:59.00").unix
sel_slews = [
    slews[i]
    for i in range(len(slews))
    if (slews[i].begin.unix > date_min and slews[i].end.unix < date_end)
]
print(f"Selected {len(sel_slews)} slews out of {len(slews)} for {dayObs}")

In [ ]:
# Check that all the selected slews fall within dayObs
print(sel_slews[0].begin.datetime64, sel_slews[-1].end.datetime64)

In [ ]:
client = EfdClient("usdf_efd")

In [ ]:
# Print slews amplitudes and speed in azimuth and elevation
# The selected slew numbers will be stored in variables slew_azi and slew_ele
# by default they contain the slews with the largest amplitudes
max_delta_azi = -99
max_delta_ele = -99
for i_slew, slew in enumerate(sel_slews):
    df_ele = getEfdData(client, "lsst.sal.MTMount.elevation", event=slew)
    df_azi = getEfdData(client, "lsst.sal.MTMount.azimuth", event=slew)

    t1 = slew.begin.datetime64
    t2 = slew.end.datetime64
    if len(df_ele) > 0:
        slew_delta_ele = df_ele["demandPosition"].max() - df_ele["demandPosition"].min()
        if slew_delta_ele > 30:
            print(
                f"Slew number: {i_slew} - Delta ele: {slew_delta_ele:.1f} degrees - Speed: {abs(df_ele['actualVelocity']).max()}",
                t1,
                t2,
            )
        if slew_delta_ele > max_delta_ele:
            max_delta_ele = slew_delta_ele
            slew_ele = i_slew
    if len(df_azi) > 0:
        slew_delta_azi = df_azi["actualPosition"].max() - df_azi["actualPosition"].min()
        if slew_delta_azi > 30:
            print(
                f"Slew number: {i_slew} - Delta azi: {slew_delta_azi:.1f} degrees - Speed: {abs(df_azi['actualVelocity']).max()}",
                t1,
                t2,
            )
        if slew_delta_azi > max_delta_azi:
            max_delta_azi = slew_delta_azi
            slew_azi = i_slew
print(
    f"Maximum amplitude slew in azimuth: {slew_azi} / {max_delta_azi:.1f} degrees - in elevation: {slew_ele} / {max_delta_ele:.1f} degrees"
)

In [ ]:
# After some investigation, we find that the slew of interest is number 24
slew_select = 24

# Add a delta_t before and after the selected slew in order to get the full sequence of events around the shaking event
delta_t = TimeDelta(300, format="sec")
start_slew = sel_slews[slew_select].begin - delta_t
end_slew = sel_slews[slew_select].end + 2 * delta_t
print(f"Selected slew - start: {start_slew.datetime64} - end: {end_slew.datetime64}")

# Create a cut to pick up the interesting time window in the VMS dataframe
sel = (vms_m1m3_data["times"] > start_slew.datetime64) & (
    vms_m1m3_data["times"] < end_slew.datetime64
)

In [ ]:
# Get TMA azimuth and elevation from EFD
df_azi = getEfdData(client, "lsst.sal.MTMount.azimuth", begin=start_slew, end=end_slew)
df_ele = getEfdData(
    client, "lsst.sal.MTMount.elevation", begin=start_slew, end=end_slew
)
min_azi = np.min(df_azi["actualPosition"])
max_azi = np.max(df_azi["actualPosition"])
min_ele = np.min(df_ele["actualPosition"])
max_ele = np.max(df_ele["actualPosition"])
speed_azi = np.max(np.abs(df_azi["actualVelocity"]))
if np.max(df_azi["actualVelocity"]) < speed_azi:
    speed_azi = -speed_azi
speed_ele = np.max(np.abs(df_ele["actualVelocity"]))
if np.max(df_ele["actualVelocity"]) < speed_ele:
    speed_ele = -speed_ele

In [ ]:
# Get accelerometer data
df_accel = getEfdData(
    client, "lsst.sal.MTM1M3.accelerometerData", begin=start_slew, end=end_slew
)

In [ ]:
# Retrieve the events occuring around the shaking event
selected = [ev for ev in events if ev.begin > start_slew and ev.end < end_slew]
selected

In [ ]:
# Get the M1M3 detailed state to check when the mirror is lowered
df_det_state = getEfdData(
    client, "lsst.sal.MTM1M3.logevent_detailedState", begin=start_slew, end=end_slew
)
df_det_state["detailedStateName"] = df_det_state["detailedState"].map(
    lambda x: MTM1M3.DetailedStates(x).name
)
df_det_state

In [ ]:
# And finally get the CSC (Commandable Sal Component) state to determine the global status of the TMA
df_sum_state = getEfdData(
    client,
    "lsst.sal.MTMount.logevent_summaryState",
    begin=start_slew,
    end=end_slew,
)
df_sum_state

We see that the TMA state is transiting from 2 (enable) to 1 (disable). We will see in the following that the "disable" state correspond approximately to the time where the shaking stopped.

In [ ]:
# Main summary plot to show the sequence of events

fig, ax = plt.subplots(8, 1, sharex=True, dpi=125, figsize=(14, 14))
ax[0].plot(vms_m1m3_data["times"][sel], vms_m1m3_data["m1m3_z_3"][sel])
ax[0].set(ylabel="accel (mg)", title=f"{vms_date} - VMS Accel x_3")
ax[1].plot(df_azi.index, df_azi["actualPosition"], label="Azimuth")
ax[1].plot(df_ele.index, df_ele["actualPosition"], label="Elevation")
ax[1].legend()
ax[1].set(ylabel="deg", title=f"{vms_date} - TMA Orientation")
ax[2].plot(df_azi.index, df_azi["actualTorque"])
ax[2].plot(df_ele.index, df_ele["actualTorque"])
ax[2].set(ylabel="N.m", title=f"{vms_date} - TMA Torque")
ax[2].legend()
ax[3].plot(df_azi.index, df_azi["actualVelocity"], label="Azimuth")
ax[3].plot(df_ele.index, df_ele["actualVelocity"], label="Elevation")
ax[3].set(ylabel="deg/s", title=f"{vms_date} - TMA Actual Speed")
ax[3].legend()
ax[4].plot(df_azi.index, df_azi["demandVelocity"], label="Azimuth")
ax[4].plot(df_ele.index, df_ele["demandVelocity"], label="Elevation")
ax[4].set(ylabel="deg/s", title=f"{vms_date} - TMA Demand Speed")
ax[4].legend()
ax[5].plot(df_accel.index, df_accel["angularAccelerationX"])
ax[5].set(ylabel="deg/s²", title=f"{vms_date} - M1M3 Accel x")
ax[5].set_ylim([-1.5, 1.5])
ax[6].plot(df_accel.index, df_accel["angularAccelerationY"])
ax[6].set(ylabel="deg/s²", title=f"{vms_date} - M1M3 Accel y")
ax[6].set_ylim([-10, 5])
ax[7].plot(df_accel.index, df_accel["angularAccelerationZ"])
ax[7].set(ylabel="deg/s²", title=f"{vms_date} - M1M3 Accel z")
ax[7].set_ylim([-1.5, 1.5])
ax[len(ax) - 1].set_xlabel("Time")
for i in range(len(ax)):
    ax[i].xaxis.set_major_formatter(DateFormatter("%H-%M-%S"))
    plt.setp(ax[i].get_xticklabels(), rotation=45)
    for ct, ev in enumerate(selected):
        if i == 0 and ct == 0:
            ax[i].axvline(ev.begin.datetime64, color="y", ls="--", label="Start slew")
            ax[i].axvline(ev.end.datetime64, color="g", ls="--", label="End slew")
            ax[0].legend()
        else:
            ax[i].axvline(ev.begin.datetime64, color="y", ls="--")
            ax[i].axvline(ev.end.datetime64, color="g", ls="--")
    for j, ind in enumerate(df_det_state.index):
        if df_det_state["detailedStateName"][ind] == "LOWERING":
            if i == 0:
                ax[i].axvline(
                    df_det_state.index[j], color="b", ls="dotted", label="Lowering"
                )
            else:
                ax[i].axvline(df_det_state.index[j], color="b", ls="dotted")
        elif df_det_state["detailedStateName"][ind] == "PARKED":
            if i == 0:
                ax[i].axvline(df_det_state.index[j], color="black", label="Parked")
            else:
                ax[i].axvline(df_det_state.index[j], color="black")
    for k, ind in enumerate(df_sum_state.index):
        if df_sum_state["summaryState"][ind] == 1:
            if i == 0:
                ax[i].axvline(
                    df_sum_state.index[k],
                    color="magenta",
                    label="CSC Disable",
                )
            else:
                ax[i].axvline(df_sum_state.index[j], color="magenta")
    if i == 0:
        ax[0].legend(loc="upper left")

fig.tight_layout()
fig.savefig(f"{plot_dir}/overview-{vms_date}-{slew_select}-.png")

## Check whether HP forces are correlated with VMS data

In [ ]:
df_hp = getEfdData(
    client, "lsst.sal.MTM1M3.hardpointActuatorData", begin=start_slew, end=end_slew
)

In [ ]:
fig, ax = plt.subplots(3, 1, sharex=True, dpi=125, figsize=(14, 8))
bx = []
for i, axis in enumerate("xyz"):
    lns1 = ax[i].plot(
        vms_m1m3_data["times"][sel],
        vms_m1m3_data[f"m1m3_{axis}_2"][sel],
        lw=0.5,
        label=f"Accel 2_{axis}",
    )
    ax[i].set(ylabel="accel (1e-3 g)")
    bx.append(ax[i].twinx())
    lns2 = bx[i].plot(
        df_hp.index, df_hp[f"f{axis}"], c="red", lw=0.5, alpha=0.7, label=f"HP f{axis}"
    )
    bx[i].set_ylabel(f"Total Force (N)")
    ax[i].grid(axis="both")
    lns = lns1 + lns2
    labs = [l.get_label() for l in lns]
    ax[i].legend(lns, labs, loc=0)

    for ct, ev in enumerate(selected):
        if ct == 0:
            ax[i].axvline(ev.begin.datetime64, color="y", ls="--", label="Start slew")
            ax[i].axvline(ev.end.datetime64, color="g", ls="--", label="End slew")
        else:
            ax[i].axvline(ev.begin.datetime64, color="y", ls="--")
            ax[i].axvline(ev.end.datetime64, color="g", ls="--")
    for j, ind in enumerate(df_det_state.index):
        if df_det_state["detailedStateName"][ind] == "LOWERING":
            ax[i].axvline(
                df_det_state.index[j], color="b", ls="dotted", label="Lowering"
            )
        elif df_det_state["detailedStateName"][ind] == "PARKED":
            ax[i].axvline(df_det_state.index[j], color="black", label="Parked")
    for k, ind in enumerate(df_sum_state.index):
        if df_sum_state["summaryState"][ind] == 1:
            ax[i].axvline(
                df_sum_state.index[k],
                color="magenta",
                label="CSC Disable",
            )

    ax[i].legend(loc="upper left")
    bx[i].legend(loc="lower left")

#    for ev in selected:
#        ax[i].axvline(ev.begin.datetime64, color="y")
ax[2].set(xlabel="Time")
ax[0].set(title=f"{vms_date}")
ax[2].xaxis.set_major_formatter(DateFormatter("%H-%M-%S"))
plt.setp(ax[i].get_xticklabels(), rotation=45)
fig.tight_layout()
fig.savefig(f"{plot_dir}/vms-HP-{vms_date}-{slew_select}-.png")

In [ ]:
# Simple Fourier analysis

begin = sel_slews[slew_select].begin
end = sel_slews[slew_select].end

fig, ax = plt.subplots(3, 3, dpi=125, figsize=(10, 8))
for c in range(3):

    subdat_sel = (vms_m1m3_data["times"] > begin.datetime64) & (
        vms_m1m3_data["times"] < end.datetime64
    )
    subdat = vms_m1m3_data.loc[subdat_sel, :]
    for j, axis in enumerate("xyz"):
        key = f"m1m3_{axis}_{c+1}"
        norm_signal = np.int16((subdat[key] / subdat[key].max()) * 32767)
        sig_fft = rfft(norm_signal)
        sample_freq = rfftfreq(
            len(subdat[key]), np.mean(np.diff(Time(subdat["times"]).unix))
        )
        power = np.abs(sig_fft) ** 2

        ax[c][j].plot(sample_freq, power, lw=1)
        ax[c][j].set_xticks(np.arange(0, 110, 10))
        ax[c][j].set(
            ylabel="Power",
            xlabel="Frequency [Hz]",
            title=f"Sensor {c+1} - axis {axis}",
        )
        yticks = ticker.MaxNLocator(4)
        ax[c][j].yaxis.set_major_locator(yticks)
        xticks = ticker.MaxNLocator(6)
        ax[c][j].xaxis.set_major_locator(xticks)
        # ax[c][j].set_ylim([0, 1e-5])
        # ax[c][j].set_ylim([0, 1e-6])
        # ax[c][j].set_yscale("log")
# fig.suptitle(f"FCU speed: {percentv}%")
date = sel_slews[slew_select].begin.datetime.replace(microsecond=0)
seqNum = sel_slews[slew_select].seqNum
speed_azi = abs(df_azi["actualVelocity"]).max()
speed_ele = abs(df_ele["actualVelocity"]).max()
fig.suptitle(
    f"{date} - seqNum: {seqNum} - Speed Azi: {speed_azi:.1f} deg/s - Speed Ele: {speed_ele:.1f} deg/s"
)
fig.tight_layout()

In [ ]:
# Here redo the same Fourer analysis plots as above but zooming on the low frequency region

begin = sel_slews[slew_select].begin
end = sel_slews[slew_select].end

fig, ax = plt.subplots(3, 3, dpi=125, figsize=(10, 8))
for c in range(3):

    subdat_sel = (vms_m1m3_data["times"] > begin.datetime64) & (
        vms_m1m3_data["times"] < end.datetime64
    )
    subdat = vms_m1m3_data.loc[subdat_sel, :]
    for j, axis in enumerate("xyz"):
        key = f"m1m3_{axis}_{c+1}"
        norm_signal = np.int16((subdat[key] / subdat[key].max()) * 32767)
        sig_fft = rfft(norm_signal)
        sample_freq = rfftfreq(
            len(subdat[key]), np.mean(np.diff(Time(subdat["times"]).unix))
        )
        power = np.abs(sig_fft) ** 2

        peaks, _ = signal.find_peaks(power, height=5.0e13, distance=10)
        t_peak = [sample_freq[ll] for ll in peaks]
        y_peak = np.full(len(t_peak), 0.8 * np.max(power))
        print(t_peak)

        ax[c][j].plot(sample_freq, power, lw=1)
        ax[c][j].plot(t_peak, y_peak, "r+", label="Peaks")
        ax[c][j].set_xticks(np.arange(0, 110, 10))
        ax[c][j].set(
            ylabel="Power",
            xlabel="Frequency [Hz]",
            title=f"Sensor {c+1} - axis {axis}",
        )
        yticks = ticker.MaxNLocator(4)
        ax[c][j].yaxis.set_major_locator(yticks)
        xticks = ticker.MaxNLocator(6)
        ax[c][j].xaxis.set_major_locator(xticks)
        ax[c][j].set_xlim([-0.5, 10])
        ax[c][j].legend()
        # ax[c][j].set_ylim([0, 1e-5])
        # ax[c][j].set_ylim([0, 1e-6])
        # ax[c][j].set_yscale("log")
# fig.suptitle(f"FCU speed: {percentv}%")
date = sel_slews[slew_select].begin.datetime.replace(microsecond=0)
seqNum = sel_slews[slew_select].seqNum
speed_azi = abs(df_azi["actualVelocity"]).max()
speed_ele = abs(df_ele["actualVelocity"]).max()
fig.suptitle(
    f"{date} - seqNum: {seqNum} - Speed Azi: {speed_azi:.1f} deg/s - Speed Ele: {speed_ele:.1f} deg/s"
)
fig.tight_layout()
fig.savefig(f"{plot_dir}/Fourier-{vms_date}-{slew_select}-.png")

From the previous plots, we see that all accelerometers / axes detect a peak at at frequency of 1.26 Hz